In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import gym
from gym import spaces
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
from torch.utils.tensorboard import SummaryWriter
import pickle

In [ ]:

def save_study(study, filename):
    with open(filename, 'wb') as f:
        pickle.dump(study, f)

def load_study(filename):
    if os.path.isfile(filename):
        with open(filename, 'rb') as f:
            study = pickle.load(f)
        return study
    return None

# Create or load the study
study_filename = 'ppo_study.pkl'
study = load_study(study_filename)
if study is None:
    study = optuna.create_study(direction='maximize')

import torch

def save_checkpoint(agent, optimizer_actor, optimizer_critic, episode, filename):
    checkpoint = {
        'agent_state_dict': agent.state_dict(),
        'optimizer_actor_state_dict': optimizer_actor.state_dict(),
        'optimizer_critic_state_dict': optimizer_critic.state_dict(),
        'episode': episode
    }
    torch.save(checkpoint, filename)

def load_checkpoint(agent, optimizer_actor, optimizer_critic, filename):
    if os.path.isfile(filename):
        checkpoint = torch.load(filename)
        agent.load_state_dict(checkpoint['agent_state_dict'])
        optimizer_actor.load_state_dict(checkpoint['optimizer_actor_state_dict'])
        optimizer_critic.load_state_dict(checkpoint['optimizer_critic_state_dict'])
        episode = checkpoint['episode']
        return episode
    else:
        return 0

In [ ]:
# Load the preprocessed data
file_path = 'input.csv'
data = pd.read_csv(file_path)

# Define headers
stock_headers = ['INFY', 'BSOFT', 'BBOX', 'ACCELYA', 'HBLPOWER', 'BOSCHLTD', 'NCC', 'AUROPHARMA', 'NATCOPHARM', 'SHRIRAMFIN', 'HINDUNILVR', 'SBIN', 'DRREDDY', 'BHARTIARTL', 'ONGC']
bond_headers = ['IN5Y', 'IN10Y']
macro_headers = ['Inflation', 'GDP', 'Unemployment', 'Repo Rate', 'Corporate Tax rate', 'IIP', 'Exchange Rate']
tech_indicators = ['SMA_20', 'EMA_20', 'EMA_50', 'RSI', 'BB_High', 'BB_Low', 'BB_Mid', 'MACD', 'MACD_Signal',
                   'MACD_Diff', 'ATR', 'Stoch', 'Stoch_Signal', 
                   'SMA_20_x_RSI', 'SMA_20_x_MACD', 'RSI_x_MACD']

# Combine all headers
features = stock_headers + bond_headers + macro_headers + tech_indicators

# Split the data into training (90%) and testing (10%) sets
split_idx = int(len(data) * 0.9)
train_data = data.iloc[:split_idx]
test_data = data.iloc[split_idx:]

# Handle missing values and infinities
train_data[features] = train_data[features].replace([np.inf, -np.inf], np.nan)
train_data[features] = train_data[features].ffill().bfill()

# State data including stocks, bonds, and macroeconomic indicators for training
state_data = train_data[features].values

# Calculate daily returns
def calculate_daily_returns(prices):
    returns = prices.pct_change().dropna()
    return returns

daily_returns = train_data[stock_headers + bond_headers].apply(calculate_daily_returns)

def ewma_volatility(returns, lambda_=0.94):
    squared_returns = returns ** 2
    ewma_variance = squared_returns.ewm(span=(2 / (1 - lambda_)) - 1, adjust=False).mean()
    ewma_volatility = np.sqrt(ewma_variance)
    return ewma_volatility

volatilities = daily_returns.apply(ewma_volatility).dropna()

In [ ]:
class ReplayBuffer:
    def __init__(self, max_size=1000000):
        self.buffer = []
        self.max_size = max_size
        self.ptr = 0

    def add(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.max_size:
            self.buffer.append((state, action, reward, next_state, done))
        else:
            self.buffer[self.ptr] = (state, action, reward, next_state, done)
        self.ptr = (self.ptr + 1) % self.max_size

    def sample(self, batch_size):
        idx = np.random.choice(len(self.buffer), batch_size)
        states, actions, rewards, next_states, dones = zip(*[self.buffer[i] for i in idx])
        return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

In [ ]:
class PortfolioEnv(gym.Env):
    def __init__(self, state_data, volatilities, stock_indices):
        super(PortfolioEnv, self).__init__()
        self.state_data = state_data
        self.volatilities = volatilities.values
        self.stock_indices = stock_indices
        self.n_steps, self.n_features = state_data.shape
        self.action_space = spaces.Box(low=0, high=1, shape=(len(stock_indices),), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_features,), dtype=np.float32)
        self.current_step = 0
        self.done = False
        self.risk_free_rate = 0.03  # Risk-free rate
        self.reward_history = []  # To keep track of reward history for normalization

    def reset(self):
        self.current_step = 0
        self.done = False
        return self.state_data[self.current_step]

    def step(self, action_weights):
        if self.current_step >= self.n_steps - 2:  # Ensure next step is within bounds
            self.done = True
            return self.state_data[self.current_step], 0, self.done, {}

        self.current_step += 1
        reward = self.calculate_reward(action_weights)
        state = self.state_data[self.current_step]
        return state, reward, self.done, {}

    def calculate_reward(self, action_weights):
        if self.current_step >= self.n_steps - 1:  # Ensure next step is within bounds
            return 0

        # Calculate returns using only the stock indices
        returns = self.state_data[self.current_step + 1, self.stock_indices] - self.state_data[self.current_step, self.stock_indices]
        portfolio_return = np.sum(action_weights * returns)
        portfolio_volatility = np.sqrt(np.sum(action_weights ** 2 * self.volatilities[self.current_step][self.stock_indices]))

        sharpe_ratio = (portfolio_return - self.risk_free_rate) / (portfolio_volatility + 1e-10)
        reward = sharpe_ratio - 0.01 * portfolio_volatility  # Added volatility penalty
        
        self.reward_history.append(reward)
        
        if len(self.reward_history) > 1:
            mean_reward = np.mean(self.reward_history)
            std_reward = np.std(self.reward_history)
            reward = (reward - mean_reward) / (std_reward + 1e-10)

        return reward

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, num_heads, output_dim):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.output_dim = output_dim
        self.head_dim = output_dim // num_heads  # Each head's dimension
        assert self.head_dim * num_heads == self.output_dim, "output_dim must be divisible by num_heads"
        self.attention_heads = nn.ModuleList([nn.Linear(input_dim, self.head_dim) for _ in range(num_heads)])

    def forward(self, x):
        outputs = [head(x) for head in self.attention_heads]
        outputs = torch.cat(outputs, dim=-1)  # Concatenate along the last dimension
        return outputs

In [ ]:
class PPOAgent(nn.Module):
    def __init__(self, state_dim, action_dim, action_space, hidden_dim=256, num_heads=4, output_dim=128, lr=0.0003, eps_clip=0.2, gamma=0.99, entropy_weight=0.01):
        super(PPOAgent, self).__init__()
        self.attention = MultiHeadAttention(state_dim, num_heads, output_dim)
        self.actor = self.build_network(output_dim, action_dim * 2, hidden_dim)  # Mean and log_std for normal distribution
        self.critic = self.build_network(output_dim, 1, hidden_dim)
        
        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=lr)
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=lr)
        
        self.scheduler_actor = optim.lr_scheduler.StepLR(self.optimizer_actor, step_size=100, gamma=0.9)
        self.scheduler_critic = optim.lr_scheduler.StepLR(self.optimizer_critic, step_size=100, gamma=0.9)
        
        self.eps_clip = eps_clip
        self.gamma = gamma
        self.entropy_weight = entropy_weight
        self.entropy_decay_rate = 0.995
        self.action_space = action_space

    def build_network(self, input_dim, output_dim, hidden_dim):
        return nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Softmax(dim=-1) if output_dim > 1 else nn.Identity()
        )

    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0)
        attention_output = self.attention(state)
        mean, log_std = self.actor(attention_output).chunk(2, dim=-1)
        log_std = torch.clamp(log_std, -20, 2)
        std = log_std.exp()
        normal = Normal(mean, std)
        action = normal.sample()
        action = torch.tanh(action)
        return action.squeeze(0).cpu().detach().numpy(), normal.log_prob(action).sum(-1), normal.entropy().sum(-1)

    def compute_returns(self, rewards, dones):
        returns = []
        R = 0
        for r, d in zip(reversed(rewards), reversed(dones)):
            R = r + self.gamma * R * (1 - d)
            returns.insert(0, R)
        return torch.FloatTensor(returns)

    def update(self, states, actions, log_probs, returns, entropies):
        states = torch.FloatTensor(np.array(states))
        actions = torch.FloatTensor(np.array(actions))
        log_probs = torch.stack(log_probs).detach()
        returns = torch.FloatTensor(returns)
        entropies = torch.stack(entropies).detach()

        attention_output = self.attention(states)
        advantages = returns - self.critic(attention_output).squeeze().detach()

        for _ in range(10):
            mean, log_std = self.actor(attention_output).chunk(2, dim=-1)
            std = log_std.exp()
            normal = Normal(mean, std)
            new_log_probs = normal.log_prob(actions).sum(-1)
            ratio = torch.exp(new_log_probs - log_probs)
            surr1 = ratio * advantages
            surr2 = torch.clamp(ratio, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            actor_loss = -torch.min(surr1, surr2).mean()

            critic_loss = advantages.pow(2).mean()
            entropy_loss = -entropies.mean() * self.entropy_weight

            # Add L2 regularization
            l2_reg = torch.tensor(0.)
            for param in self.actor.parameters():
                l2_reg += torch.norm(param)
            loss = actor_loss + 0.5 * critic_loss + entropy_loss + l2_reg

            self.optimizer_actor.zero_grad()
            self.optimizer_critic.zero_grad()
            loss.backward(retain_graph=True)
            self.optimizer_actor.step()
            self.optimizer_critic.step()

            # Logging for debugging
            print(f"Update Step, Actor Loss: {actor_loss.item()}, Critic Loss: {critic_loss.item()}, Entropy Loss: {entropy_loss.item()}, Total Loss: {loss.item()}")

        # Step the learning rate schedulers
        self.scheduler_actor.step()
        self.scheduler_critic.step()

        # Clear cache after each update
        torch.cuda.empty_cache()

    def decay_entropy_weight(self):
        self.entropy_weight *= self.entropy_decay_rate

In [ ]:
# Setup TensorBoard and Early Stopping
writer = SummaryWriter()

class EarlyStopping:
    def __init__(self, patience=100, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_score = None
        self.counter = 0

    def step(self, score):
        if self.best_score is None:
            self.best_score = score
            return False
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.best_score = score
            self.counter = 0
        return False

early_stopping = None  # EarlyStopping(patience=10)

In [ ]:
def train_ppo(env, agent, episodes=1000, writer=None, save_interval=10, checkpoint_path="ppo_checkpoint.pth"):
    start_episode = load_checkpoint(agent, agent.optimizer_actor, agent.optimizer_critic, checkpoint_path)
    scheduler = optim.lr_scheduler.StepLR(agent.optimizer_actor, step_size=3, gamma=0.9)  # Learning rate scheduler
    for ep in range(start_episode, episodes):
        state = env.reset()
        states, actions, log_probs, rewards, dones, entropies = [], [], [], [], [], []
        done = False
        while not done:
            action, log_prob, entropy = agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            states.append(state)
            actions.append(action)
            log_probs.append(log_prob)
            rewards.append(reward)
            dones.append(done)
            entropies.append(entropy)
            state = next_state

        returns = agent.compute_returns(rewards, dones)
        agent.update(states, actions, log_probs, returns, entropies)

        episode_return = sum(rewards)
        print(f"Episode {ep}, Return: {episode_return}")

        if writer:
            writer.add_scalar("Return", episode_return, ep)

        # Save checkpoint
        if ep % save_interval == 0:
            save_checkpoint(agent, agent.optimizer_actor, agent.optimizer_critic, ep, checkpoint_path)

        # Update learning rate and decay entropy weight
        scheduler.step()
        agent.decay_entropy_weight()

    print(f"Final Weights (PPO): {agent.latest_action}")
    print(f"Training is complete")

In [ ]:
def save_agent(agent, filename):
    torch.save(agent.state_dict(), filename)

def load_agent(agent, filename):
    if os.path.isfile(filename):
        checkpoint = torch.load(filename)
        agent.load_state_dict(checkpoint, strict=False)

In [ ]:
import optuna
import torch

study_filename = 'ppo_study.pkl'
checkpoint_path = 'ppo_checkpoint.pth'

# Create or load the study
study = load_study(study_filename)
if study is None:
    study = optuna.create_study(direction='maximize')

def objective(trial):
    # Define hyperparameters
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    eps_clip = trial.suggest_float('eps_clip', 0.1, 0.3)
    gamma = trial.suggest_float('gamma', 0.9, 0.99)
    entropy_weight = trial.suggest_float('entropy_weight', 1e-3, 1e-2, log=True)
    num_heads = trial.suggest_int('num_heads', 2, 8)
    output_dim_base = trial.suggest_int('output_dim_base', 16, 64)

    output_dim = output_dim_base * num_heads

    agent = PPOAgent(state_dim, action_dim, env.action_space, hidden_dim, num_heads, output_dim, lr, eps_clip, gamma, entropy_weight)

    # Train the agent
    def train_ppo_trial(env, agent, episodes=30):
        total_rewards = []
        for ep in range(episodes):
            state = env.reset()
            states, actions, log_probs, rewards, dones, entropies = [], [], [], [], [], []
            done = False
            step = 0
            while not done:
                step += 1
                action, log_prob, entropy = agent.select_action(state)
                next_state, reward, done, _ = env.step(action)
                states.append(state)
                actions.append(action)
                log_probs.append(log_prob)
                rewards.append(reward)
                dones.append(done)
                entropies.append(entropy)
                state = next_state

                if step % 100 == 0:
                    print(f"Episode {ep}, Step {step}, Reward: {reward}")

            returns = agent.compute_returns(rewards, dones)
            agent.update(states, actions, log_probs, returns, entropies)

            episode_return = sum(rewards)
            total_rewards.append(episode_return)
            trial.report(episode_return, ep)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return sum(total_rewards)

    total_rewards = train_ppo_trial(env, agent)
    return float(total_rewards)

# Optimize the study
study.optimize(objective, n_trials=100)

# Save the study
save_study(study, study_filename)

# Best hyperparameters
best_params = study.best_params
print("Best hyperparameters found were: ", best_params)

# Initialize the PPO agent with the best hyperparameters
ppo_agent = PPOAgent(
    state_dim,
    action_dim,
    env.action_space,
    hidden_dim=best_params['hidden_dim'],
    num_heads=best_params['num_heads'],
    output_dim=best_params['output_dim_base'] * best_params['num_heads'],
    lr=best_params['lr'],
    eps_clip=best_params['eps_clip'],
    gamma=best_params['gamma'],
    entropy_weight=best_params['entropy_weight'],
    regularization_weight=0.0001  # Light regularization
)

# Train PPO agent
num_episodes = 500  # Define the number of episodes you want to train for
train_ppo(env, ppo_agent, episodes=num_episodes, writer=writer, save_interval=10, checkpoint_path=checkpoint_path)

# Save trained PPO model
save_agent(ppo_agent, 'ppo_agent_final.pth')
print("Training complete. PPO model saved.")

In [ ]:
import cvxpy as cp
import matplotlib.pyplot as plt

# Load the CSV file for MVO
mvo_file_path = 'MVO.csv'
mvo_data = pd.read_csv(mvo_file_path)

# Define stock headers (assuming they are the first 15 columns)
stock_headers = mvo_data.columns[1:16]  # Exclude the Date column

# Calculate daily returns for MVO
daily_returns_mvo = mvo_data[stock_headers].pct_change().dropna()

# Calculate expected returns and covariance matrix for MVO using raw stock data
expected_returns_mvo = daily_returns_mvo.mean().values
cov_matrix_mvo = daily_returns_mvo.cov().values

# Mean-Variance Optimization (MVO) function
def mean_variance_optimization(expected_returns, cov_matrix, risk_aversion=1):
    n = len(expected_returns)
    w = cp.Variable(n)
    objective = cp.Maximize(expected_returns.T @ w - risk_aversion * cp.quad_form(w, cov_matrix))
    constraints = [cp.sum(w) == 1, w >= 0]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    # Ensure the weights sum to 1
    weights = w.value
    weights /= np.sum(weights)
    
    return weights

# Calculate MVO weights
mvo_weights = mean_variance_optimization(expected_returns_mvo, cov_matrix_mvo)
print("MVO Weights:", mvo_weights)
print("Sum of MVO Weights:", np.sum(mvo_weights))

In [ ]:
# Function to combine PPO weights with MVO weights
def combine_weights(ppo_weights, mvo_weights, alpha):
    combined_weights = (1 - alpha) * ppo_weights + alpha * mvo_weights
    return combined_weights

# Function to sweep alpha and evaluate combined weights
def evaluate_combined_weights(env, ppo_agent, alpha_values):
    results = []
    for alpha in alpha_values:
        state = env.reset()
        done = False
        episode_return = 0
        while not done:
            ppo_weights, _, _ = ppo_agent.select_action(state)
            # Only take the stock weights from the PPO agent
            ppo_stock_weights = ppo_weights[:len(stock_headers)]
            combined_weights = combine_weights(ppo_stock_weights, mvo_weights, alpha)
            # Ensure the action weights include only the stock weights for the environment step
            next_state, reward, done, _ = env.step(combined_weights[:len(stock_headers)])
            episode_return += reward
            state = next_state
        results.append((alpha, episode_return))
        print(f"Alpha: {alpha}, Episode Return: {episode_return}")
    return results

# Alpha values to sweep from 0 to 1
alpha_values = np.linspace(0, 1, 11)

# Evaluate combined weights
results = evaluate_combined_weights(env, ppo_agent, alpha_values)

# Save results to visualize later
results_df = pd.DataFrame(results, columns=['Alpha', 'Episode Return'])
results_df.to_csv('combined_weights_results.csv', index=False)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(results_df['Alpha'], results_df['Episode Return'], marker='o')
plt.title('Performance of Combined Weights for Different Alpha Values')
plt.xlabel('Alpha')
plt.ylabel('Episode Return')
plt.grid(True)
plt.savefig('combined_weights_performance.png')
plt.show()

In [ ]:
# Model evaluation
def calculate_performance_metrics(returns):
    metrics = {}
    metrics['ROI'] = np.sum(returns)
    metrics['Sharpe Ratio'] = np.mean(returns) / (np.std(returns) + 1e-10)
    downside_returns = returns[returns < 0]
    metrics['Sortino Ratio'] = np.mean(returns) / (np.std(downside_returns) + 1e-10)
    cum_returns = np.cumsum(returns)
    drawdown = np.max(cum_returns) - cum_returns
    metrics['Maximum Drawdown'] = np.max(drawdown)
    metrics['Calmar Ratio'] = metrics['ROI'] / (metrics['Maximum Drawdown'] + 1e-10)
    return metrics

# Backtesting
def backtest(env, agent, alpha_values):
    backtest_results = {}
    for alpha in alpha_values:
        state = env.reset()
        done = False
        returns = []
        while not done:
            ppo_weights, _, _ = agent.select_action(state)
            ppo_stock_weights = ppo_weights[:len(stock_headers)]
            combined_weights = combine_weights(ppo_stock_weights, mvo_weights, alpha)
            next_state, reward, done, _ = env.step(combined_weights)
            returns.append(reward)
            state = next_state
        metrics = calculate_performance_metrics(np.array(returns))
        backtest_results[alpha] = metrics
        print(f"Alpha: {alpha}, Metrics: {metrics}")
    return backtest_results

# Perform backtesting
backtest_results = backtest(env, ppo_agent, alpha_values)

# Save backtest results
backtest_df = pd.DataFrame(backtest_results).T
backtest_df.to_csv('backtest_results.csv', index=False)

# Plot backtest results
backtest_df.plot(kind='bar', figsize=(12, 8))
plt.title('Backtest Performance Metrics for Different Alpha Values')
plt.xlabel('Alpha')
plt.ylabel('Metrics')
plt.legend(loc='best')
plt.savefig('backtest_performance_metrics.png')
plt.show()